In [1]:
import torch
import torch.optim as optim
from scipy.sparse.linalg import eigsh
import numpy as np
import yastn
# import yastn.Tensor as ytensor
import yastn.tn.fpeps as fpeps
import yastn.operators as op_mod
from tqdm import tqdm
from functions_fpeps import *
from functions_ed import *
import matplotlib.pyplot as plt

NUM_THREADS = 8 
torch.set_num_threads(NUM_THREADS)

In [ ]:
L = 2
L_x, L_y = L, L
t = 1.0
U = 10.0
mu = 0.0
learning_rate = 1
n_var_steps = 1000
ntu_steps = 10
D_target = 4
chi = 5 * D_target
output_no = 8
max_ctm_sweeps = 20
tol_ctm = 1e-10
dtau = 0.05

config_kwargs = {
    "backend": "torch",
    "default_dtype": "float128",
    "device": "cpu"
}

In [3]:
ops = yastn.operators.SpinfulFermions(sym="U1xU1", **config_kwargs)

I = ops.I()
c_up, cdag_up, n_up = ops.c('u'), ops.cp('u'), ops.n('u')
c_dn, cdag_dn, n_dn = ops.c('d'), ops.cp('d'), ops.n('d')

state_up = ops.vec_n((1, 0))
state_dn = ops.vec_n((0, 1))

geometry = fpeps.SquareLattice(dims=(L_x, L_y), boundary="obc")

vectors = {}
for (x, y) in geometry.sites():
    if (x + y) % 2 == 0:
        vectors[(x, y)] = state_up
    else:
        vectors[(x, y)] = state_dn

psi = fpeps.product_peps(geometry=geometry, vectors=vectors)

g_hop_up = fpeps.gates.gate_nn_hopping(t, dtau / 2, I, c_up, cdag_up)
g_hop_dn = fpeps.gates.gate_nn_hopping(t, dtau / 2, I, c_dn, cdag_dn)
g_loc = fpeps.gates.gate_local_Coulomb(mu, mu, U, dtau / 2, I, n_up, n_dn)

gates = fpeps.gates.distribute(
    geometry,
    gates_nn=[g_hop_up, g_hop_dn],
    gates_local=g_loc
)

In [4]:
env_ntu = fpeps.EnvNTU(psi, which="NN++")
opts_svd_ntu = {
    "D_total": D_target,
}
infoss = []
for step in tqdm(range(ntu_steps)):
    infos = fpeps.evolution_step_(env_ntu, gates, opts_svd=opts_svd_ntu)
    infoss.append(infos)

100%|██████████| 10/10 [00:20<00:00,  2.10s/it]


In [5]:
for site in geometry.sites():
    psi[site].requires_grad_(True)
opts_svd_ctm = {
    "D_total": chi,
    "tol": tol_ctm
}
energy_yastn = compute_energy_ctm(psi, opts_svd_ctm, max_ctm_sweeps, I, n_up, n_dn, cdag_up, cdag_dn, c_up, c_dn, U, t)